In [2]:
%%capture
!pip install tensorboardX simpletransformers sklearn

In [3]:
5

5

In [4]:
!cat /usr/local/cuda/version.txt
!pip install pydash
!pip install torch==1.6.0
#conda install pytorch==1.1.0 torchvision==0.3.0 cudatoolkit=10.0 -c pytorch
#https://pytorch.org/get-started/previous-versions/

CUDA Version 10.0.130
  Using cached pydash-5.1.0-py3-none-any.whl (84 kB)
     |████████████████████████████████| 748.8 MB 5.9 kB/s              
  Attempting uninstall: torch
    Found existing installation: torch 1.4.0
    Uninstalling torch-1.4.0:
      Successfully uninstalled torch-1.4.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
fastai 1.0.61 requires nvidia-ml-py3, which is not installed.


In [153]:
!rm -rf outputs

In [142]:
from simpletransformers.classification import ClassificationModel, ClassificationArgs
import pandas as pd
import logging
import sklearn
from tqdm import tqdm
import pandas as pd

df = pd.read_csv("downloads/train_june20_all.csv")
len(df)

44632

In [143]:
df.source.unique()

array(['acl_cambridge', 'predicts_acl_pm2500_vocabed',
       'Oct1_clinical_studies_pm', 'oct3_labels', 'labels_oct7',
       ' limitation|problem ... with ... l=be that', 'a ... drawback of',
       'a {negative_adjectives} ...  {problem_nouns}', 'pm_OBJECTIVE',
       'pm_METHODS', 'pm_RESULTS', 'pm_CONCLUSIONS', 'biology1k_050522',
       'psych1k_100522', 'june152022_labels_envgeosoc',
       'june152022_labels_matscichem'], dtype=object)

In [144]:
#downsample medical data in training set
# df = df[df["source"].notin(["oct3_labels", "june152022_labels_matscichem", 
#                            "june152022_labels_envgeosoc", "labels_oct7", 
#                            "psych1k_100522", "biology1k_050522", "predicts_acl_pm2500_vocabed", 
#                            'pm_CONCLUSIONS', 'biology1k_050522'])]

In [145]:
df = df[df["source"].isin(['acl_cambridge', 'predicts_acl_pm2500_vocabed',
       'Oct1_clinical_studies_pm', 'oct3_labels', 'labels_oct7',
       ' limitation|problem ... with ... l=be that', 'a ... drawback of',
       'a {negative_adjectives} ...  {problem_nouns}', 'pm_OBJECTIVE',
       'pm_METHODS', 'pm_RESULTS', 'pm_CONCLUSIONS', 'biology1k_050522',
       'psych1k_100522',])]

In [146]:
df[df.labels == 0]

,text,doi,journalName,title,fields,source,labels
496,Obtained lexical entries are guaranteed to con...,NaN,NaN,NaN,Computer Science,acl_cambridge,0
497,Sections 3 and 4 describe the features induced...,NaN,NaN,NaN,Computer Science,acl_cambridge,0
498,"Until the system finds a multiple entry , it b...",NaN,NaN,NaN,Computer Science,acl_cambridge,0
499,-LRB- dictionary -RRB- No price for the new sh...,NaN,NaN,NaN,Computer Science,acl_cambridge,0
500,Variation across languages can to a large exte...,NaN,NaN,NaN,Computer Science,acl_cambridge,0
...,...,...,...,...,...,...,...
42297,Widen Implementation of Entrepreneurship Educa...,10.21831/ECONOMIA.V8I1.823,Jurnal Economia,Perluasan Implementasi Pendidikan Kewirausahaa...,Psychology,psych1k_100522,0
42300,Purine Nucleoside Mediated Neuroprotection in ...,NaN,NaN,Purine Nucleoside Mediated Neuroprotection in ...,Psychology,psych1k_100522,0
42304,The problem in this research is there a possit...,NaN,NaN,Pengaruh Program Remedial Terhadap Prestasi Be...,Psychology,psych1k_100522,0
42308,Mobbing is actually a social problem.,NaN,NaN,MOBING NA RADNOM MJESTU,Psychology,psych1k_100522,0


In [147]:
df= df[["text", "labels"]]

In [148]:
df =df.dropna()
df = df.rename(columns={"labels": "label"})

In [149]:
df= df[["text", "label"]]
df.label = df.label.map(int)

In [150]:
df

,text,label
0,The difficulty with this task lies in the fact...,1
1,The problem with rich annotations is that they...,1
2,"As a consequence , when adapting existing meth...",1
3,The second problem of traditional word alignme...,1
4,The main drawback of these systems is that the...,1
...,...,...
42308,Mobbing is actually a social problem.,0
42309,Students starting off on a tertiary education ...,1
42310,Many individuals with vision impairment experi...,1
42311,Considering China is facing a precipitous decl...,0


In [154]:
logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

df2 = df.sample(frac=1)
# Train and Evaluation data needs to be in a Pandas Dataframe of two columns. 
#The first column is the text with type str, and the second column is the label with type int.
size = len(df2)
train_cutoff = round(size*0.8)
train_df= df2[:train_cutoff]
eval_df = df2[train_cutoff:]

model_args = ClassificationArgs(save_steps=-1, save_model_every_epoch=False, sliding_window=True)

model = ClassificationModel('distilbert', 'distilbert-base-uncased',args=model_args, use_cuda=True) # You can set class weights by using the optional weight argument

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.weight', 'vocab_transform.bias', 'vocab_projector.bias', 'vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_layer_norm.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.bias', 'pre_classifier.weight', 'classi

In [155]:
# Train the model
model.train_model(train_df, overwrite_output_dir=True)
# Evaluate the model, u
result, model_outputs, wrong_predictions = model.eval_model(eval_df, acc=sklearn.metrics.accuracy_score)

/home/ec2-user/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/simpletransformers/classification/classification_model.py:602: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."
INFO:simpletransformers.classification.classification_model: Converting to features started. Cache is not used.
INFO:simpletransformers.classification.classification_model: Sliding window enabled


  0%|          | 0/33850 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_model: 33867 features created from 33850 samples.
/home/ec2-user/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Running Epoch 0 of 1:   0%|          | 0/4234 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_model: Training of distilbert model complete. Saved to outputs/.
/home/ec2-user/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/simpletransformers/classification/classification_model.py:1443: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."
INFO:simpletransformers.classification.classification_model: Converting to features started. Cache is not used.
INFO:simpletransformers.classification.classification_model: Sliding window enabled


  0%|          | 0/8463 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_model: 8463 features created from 8463 samples.


Running Evaluation:   0%|          | 0/1059 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_model:{'mcc': 0.960362463847609, 'tp': 4438, 'tn': 3858, 'fp': 72, 'fn': 95, 'acc': 0.9802670447831738, 'eval_loss': 0.07107115179028013}


In [156]:
result

{'mcc': 0.960362463847609,
 'tp': 4438,
 'tn': 3858,
 'fp': 72,
 'fn': 95,
 'acc': 0.9802670447831738,
 'eval_loss': 0.07107115179028013}

In [15]:
import pandas as pd

In [172]:
dfx = pd.read_csv("downloads/16M_small_alldomains_clean.csv",  lineterminator='\n', error_bad_lines=False)

In [173]:
dfx[dfx["text"] == "nan"]

,text,doi,fields


In [174]:
# dfx = dfx[dfx.fields.isin(['Environmental Science', 'Material Science', 'Psychology', 'Geology', 'Biology', 'Materials Science'])]
dfx = dfx[dfx.fields.isin(['Medicine', 'Biology'])]


In [175]:
dfx["text"] = dfx["text"].astype(str)
len(dfx)


In [177]:
dfx.fields.value_counts(normalize=True) * 100, len(dfx)

(Medicine    69.823463
 Biology     30.176537
 Name: fields, dtype: float64,
 5771928)

In [178]:
#dfx =dfx.groupby('fields').apply(lambda x: x.sample(10000))
#dfx.to_csv("downloads/sample_bioadjacent600k.csv", index = False, header=True)

In [179]:
p1 = "downloads/predicts_21June_5M_biomed.txt"
predicts_path = 'downloads/predicts_5M_medicine.txt'

In [181]:
def stepwise(step, df, f):
    l = []
    for idx in range(step, len(df)+step-1, step):
        print(idx-step, idx)# dataframe[idx-step:idx])
        sl = df[idx-step:idx]
        batch = model.predict(list(sl))
        l = l + list(batch[0])
        with open(predicts_path, 'w') as f:
            for item in l:
                f.write("%s\n" % item)
    return l

In [ ]:
predicts = stepwise(50000, dfx["text"], model.predict)

INFO:simpletransformers.classification.classification_model: Converting to features started. Cache is not used.
INFO:simpletransformers.classification.classification_model: Sliding window enabled


0 50000


  0%|          | 0/50000 [00:00<?, ?it/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (518 > 512). Running this sequence through the model will result in indexing errors
INFO:simpletransformers.classification.classification_model: 50000 features created from 50000 samples.


  0%|          | 0/6272 [00:00<?, ?it/s]

...

INFO:simpletransformers.classification.classification_model: Converting to features started. Cache is not used.
INFO:simpletransformers.classification.classification_model: Sliding window enabled


2150000 2200000


  0%|          | 0/50000 [00:00<?, ?it/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (1177 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (518 > 512). Running this sequence through the model will result in indexing errors
INFO:simpletransformers.classification.classification_model: 50000 features created from 50000 samples.


  0%|          | 0/6273 [00:00<?, ?it/s]

In [188]:
5

5

In [189]:
len(predicts)

5771928

In [192]:
len([p for p in predicts if p==1])

739184

In [45]:
with open(p1, 'r') as f:
    p1 = f.read().split()#f.write("%s\n" % item)
                
preds = p1 + p2
len([p for p in preds if p == "1"])

In [193]:
dfx["predicts"] = predicts

## dfx = dfx[:3000000]
dfx["predicts"] = preds

In [71]:
dfx["predicts"] = dfx["predicts"].map(int)

/home/ec2-user/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [197]:
dfx = dfx[dfx["predicts"]==1]


In [198]:
dfx=dfx[dfx["doi"].notna()]

In [199]:
dfo = pd.read_csv("downloads/train_june20_all.csv")
dd = dfx[~dfx["doi"].isin(list(dfo["doi"]))]
dfx=dd

In [200]:
dfx["fields"].value_counts()

Medicine    558280
Biology     175680
Name: fields, dtype: int64

In [201]:
len(dfx)

733960

In [202]:
#dr= dfx.iloc[::-1]
dfx.to_csv("downloads/June22_biomedicine_730k_final.csv", index = False, header=True)

### Scraps

In [ ]:
df2 = dfx[dfx["fields"].isin(["Sociology", "Geology", "Environmental Science"])]
df2.to_csv("downloads/underrepresented_domains_predicts.csv", index = False, header=True)
df2 = dfx[dfx["fields"].isin(["Materials Science", "Psychology", "Chemistry"])]
df2.to_csv ("downloads/mid_represented_domains_predicts.csv", index = False, header=True)
#check that not in final110k_1605
dfx.to_csv("downloads/pos_preds_12062022.csv", index = False, header=True)
dfx[dfx["fieldsOfStudy"].isin(['Environmental Science', 'Sociology', 'Geology', 'Biology', 'Materials Science', 'Chemistry'])]

In [181]:
dt = pd.read_csv("core_pos.csv")
#d[d["first_sent"].str.contains(re.escape("|".join(d["authors"][2:-2].replace("'", "").split(","))).strip().replace('\\', ''))]
def formatStr (x):
    x = eval(x)
    x = [re.escape(v) for v in x]
    return "|".join(x)

regexs = d.authors.apply(formatStr)
matches = pd.Series()
for i, r in tqdm(d.iterrows()):
    matches.loc[i] = len(re.findall(regexs.loc[i], d.first_sent.loc[i])) > 0
dc["authors"] = dc["authors"].map(eval)
dc = dc.drop(columns=["fullText"])
dc.to_csv("core_pos_parsed.csv", index = False, header=True)
tt.to_csv("core_pos_min.csv", index = False, header=True)

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:8: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
44962it [00:59, 760.29it/s] 


distilbert
'mcc': 0.7621432780395265, 'tp': 167, 'tn': 480, 'fp': 39, 'fn': 30, 'acc': 0.9036312849162011, 'eval_loss': 0.24669100075132316}

